In [ ]:
import diffusers

In [ ]:
import os
import numpy as np
import torch


In [ ]:
target_data_list = []
target_file_dir = "E:\D1\diffusion/my_models\my_model_data\hr_images3"
target_file_list = os.listdir(target_file_dir)
for file in target_file_list:
    file_path = os.path.join(target_file_dir, file)
    data = np.load(file_path)#36,64,64
    target_data_list.append(data)

target_data = torch.from_numpy(np.stack(target_data_list).astype(np.float32))#340,36,64,64
data_mask = ~torch.isnan(target_data)
target_data = torch.unsqueeze(target_data, dim=2).view(-1, 1, 64, 64)#12240,1,64,64
np.save('E:/D1/diffusion/my_models/my_model_data/picture/real4.npy', target_data.squeeze().cpu().numpy())

In [ ]:

import torch
from diff_trainer import extract

# 测试 extract 方法

# 准备测试数据
v = torch.tensor([1.0, 2.0, 3.0, 4.0, 5.0])
t = torch.tensor([2])
x_shape = (32, 5 ,64, 64)

# 调用 extract 方法
result = extract(v, t, x_shape)

# 验证结果
expected_result = torch.tensor([3.0]).view([1] + [1] * (len(x_shape) - 1))
print(result)
print(expected_result)
assert torch.allclose(result, expected_result)




In [ ]:
import sys
print(sys.executable)

In [3]:
import torch
import numpy as np
from diff_trainer import save_image
from diff_trainer import extract
from matplotlib import pyplot as plt
import diffusers

ModuleNotFoundError: No module named 'torch'

In [ ]:
bata1 = 0.0001
bataT = 0.02
T = 1000

batas = diffusers.schedulers.scheduling_ddpm.betas_for_alpha_bar(100, alpha_transform_type='cosine')
batas = diffusers.schedulers.scheduling_ddpm.rescale_zero_terminal_snr(batas)
plt.plot(batas)
plt.show()


In [ ]:
bata1 = 0.0001
bataT = 0.02
T = 1000

batas = torch.linspace(bata1, bataT, T)
alphas = 1-batas
alphas_bar = torch.cumprod(alphas, dim=0)#计算alpha的累计乘积

alphas_bar_prev = torch.nn.functional.pad(alphas_bar, [1,0], value=1)[:T]

sqrt_alphas_bar = torch.sqrt(alphas_bar)
sqrt_1_alpha_bar = torch.sqrt(1. - alphas_bar)
coeff1 = torch.sqrt(1. / alphas)
coeff2 = coeff1 * (1. - alphas) / torch.sqrt(1. - alphas_bar)
posterior_var = batas * (1. - alphas_bar_prev)/(1. - alphas_bar)

In [ ]:
plt.plot(coeff1)
plt.show()

In [ ]:
plt.plot(coeff2)
plt.show()

In [ ]:
x0 = np.load('E:/D1/diffusion/my_models/my_model_data/picture/real_target2.npy')
noise = np.load('E:/D1/diffusion/my_models/my_model_data/picture/data_noise2.npy')
condition = np.load('E:/D1/diffusion/my_models/my_model_data/picture/condition2.npy')
eval = np.load('E:/D1/diffusion/my_models/my_model_data/picture/eval_x02.npy')
x0 = torch.tensor(x0).unsqueeze(0).unsqueeze(0)
noise = torch.tensor(noise).unsqueeze(0).unsqueeze(0)
condition = torch.tensor(condition).unsqueeze(0)
eval = torch.tensor(eval).unsqueeze(0).unsqueeze(0)

In [ ]:
save_image(x0, name="original input target", show=True)

In [ ]:
save_image(noise, name="noise", show=True)

In [ ]:
# t = torch.randint(int(T), size=(x0.shape[0],), device=x0.device)
adding_t = torch.tensor([999])
# noise = torch.randn_like(x0.float(), device=x0.device)/3
A = extract(sqrt_alphas_bar, adding_t, x0.shape)
B = extract(sqrt_1_alpha_bar, adding_t, x0.shape)
xT = A * x0 + B * noise#当t=100时，xt = 0.91 x0 + 0.42 xnoise
save_image(xT, name='noised image', show=True)

In [ ]:
plt.plot(vars)
plt.show()

In [ ]:
# xt = torch.randn_like(xT)
xt = xT
for time_step in reversed(range(100)):
    t = torch.tensor([time_step])
    eps = noise
    c1 = extract(coeff1, t, xt.shape)
    c2 = extract(coeff2, t, xt.shape)

    if time_step > 0:
        noise_z = torch.rand_like(xt.float())
    else:
        noise_z = 0
    vars = torch.cat([posterior_var[1:2], batas])
    var = extract(vars, t, xt.shape)
    xt_1 = (c1 * xt - 1*c2 * noise) + 0 * noise_z
    xt = xt_1

    print(f'timesetp: {time_step},c1: {c1},c2: {c2}, mean: {xt.mean()}, var: {xt.var()}')
rex_circle = xt
save_image(rex_circle, name='re', show=True)

In [ ]:
xt = xT
c1 = extract(coeff1, t, xt.shape)
c2 = extract(coeff2, t, xt.shape)
# c1 = torch.tensor([[[[1.006]]]])
# c2 = torch.tensor([[[[0.99]]]])
rex_str = c1 * xt - c2 * noise #当t=100时, rex = 1 xt - 0 noise
save_image(rex_str, name='re3', show=True)